## Content-based Image Retrieval

### Summary
In this notebook, we will follow the [tutorial](https://blog.sicara.com/keras-tutorial-content-based-image-retrieval-convolutional-denoising-autoencoder-dc91450cc511) which introduce the content-based image retrieval (CBIR) system. This system enable to find similar images to a query image among an image dataset. The CBIR system will be built on MNIST dataset. The CBIR system will be based on convolutional denoising autoencoder. It is a class of unsupervised deep learning algorithms.

The key of the system is how to extract features to annotate images. Examples:
* histogram of colours to define colors
* histogram of oriented gradients to define shapes

In [1]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
np.random.seed(42)

#### Denoising autoencoder for content-based image retrieval

In [3]:
from keras.layers import Input, Conv2D, MaxPooling2D
from keras.layers import UpSampling2D, ZeroPadding2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.datasets import mnist

Using TensorFlow backend.


In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') /  255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

noise_factor = 0.5 
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
y_test_noisy = np.clip(x_test_noisy, 0., 1.)

define the architect of neural network

In [8]:
def train_model():
    input_img = Input(shape=(28, 28, 1))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name='encoder')(x)
    
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    
    autoencoder.fit(x_train_noisy, 
                    x_train, 
                    epochs=20, 
                    batch_size=128, 
                    shuffle=True, 
                    validation_data=(x_test_noisy, x_test))
    
    autoencoder.save('autoencoder.h5')

In [ ]:
train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 121s - loss: 0.2716 - val_loss: 0.2249
Epoch 2/20
60000/60000 [==============================] - 105s - loss: 0.1958 - val_loss: 0.2347
Epoch 3/20
60000/60000 [==============================] - 875s - loss: 0.1806 - val_loss: 0.2445
Epoch 4/20
60000/60000 [==============================] - 131s - loss: 0.1721 - val_loss: 0.2172
Epoch 5/20
60000/60000 [==============================] - 124s - loss: 0.1656 - val_loss: 0.2339
Epoch 6/20
60000/60000 [==============================] - 150s - loss: 0.1611 - val_loss: 0.2249
Epoch 7/20
60000/60000 [==============================] - 127s - loss: 0.1575 - val_loss: 0.1826
Epoch 8/20
60000/60000 [==============================] - 124s - loss: 0.1540 - val_loss: 0.2061: 0
Epoch 9/20
33152/60000 [===============>..............] - ETA: 46s - loss: 0.1520